In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import MonthEnd
from numerize.numerize import numerize
import streamlit as st

In [2]:
import pandas as pd

destino_concluido = r"C:\Users\pcp\OneDrive\#jeff\Dev\Analise_Producao\dados\relatorio_intranet_concluido.csv"
destino_pendente = r"C:\Users\pcp\OneDrive\#jeff\Dev\Analise_Producao\dados\relatorio_intranet_pendente.csv"
destino_prod_forno = r"C:\Users\pcp\OneDrive\#jeff\Dev\Analise_Producao\dados\relatorio_forno.csv"


In [3]:
df_concluido = pd.read_csv(destino_concluido)

df_concluido['Conclusão'] = pd.to_datetime(df_concluido['Conclusão'], errors='coerce')
df_concluido['Entrada'] = pd.to_datetime(df_concluido['Entrada'], errors='coerce')

df_pendente = pd.read_csv(destino_pendente)
df_pendente['Conclusão'] = pd.to_datetime(df_pendente['Conclusão'], errors='coerce')
df_pendente['Entrada'] = pd.to_datetime(df_pendente['Entrada'], errors='coerce')

df_forno = pd.read_csv(destino_prod_forno)
df_forno['Data'] = pd.to_datetime(df_forno['Data'], errors='coerce')
df_forno['data2'] = pd.to_datetime(df_forno['data2'], errors='coerce')


In [4]:
df_resumo_forno = df_forno.groupby('data2')['Área (m2)'].sum().reset_index()
df_resumo_forno.rename(columns={'data2': 'Data', 'Área (m2)':'Área'}, inplace=True)

df_resumo_concluido = df_concluido.groupby('Entrada')['Pedido Área'].sum().reset_index()
df_resumo_concluido.rename(columns={'Entrada': 'Data', 'Pedido Área':'Área'}, inplace=True)

df_resumo_pendente = df_pendente.groupby('Entrada')['Pedido Área'].sum().reset_index()
df_resumo_pendente.rename(columns={'Entrada': 'Data', 'Pedido Área':'Área'}, inplace=True)

df_resumo_total01 = pd.merge(df_resumo_concluido, df_resumo_pendente, on='Data', how='outer')
df_resumo_total01.fillna(0, inplace=True)

df_resumo_total02 = pd.merge(df_resumo_total01, df_resumo_forno, on='Data', how='outer')
df_resumo_total02.fillna(0, inplace=True)

df_resumo_total02.rename(columns={'Área_x': 'Pedidos concluidos', 'Área_y':'Pedidos_pendentes', 'Área':'Forno'}, inplace=True)
df_resumo_total02['Total Pedidos'] = df_resumo_total02['Pedidos concluidos'] + df_resumo_total02['Pedidos_pendentes']

df_resumo_total02

,Data,Pedidos concluidos,Pedidos_pendentes,Forno,Total Pedidos
0,2025-01-02,118.45,0.00,0.00,118.45
1,2025-01-03,277.52,0.00,0.00,277.52
2,2025-01-06,903.08,0.00,653.87,903.08
3,2025-01-07,929.60,0.00,765.50,929.60
4,2025-01-08,982.51,0.00,416.88,982.51
...,...,...,...,...,...
74,2025-03-26,71.85,433.06,1279.12,504.91
75,2025-03-27,7.00,472.93,902.14,479.93
76,2025-03-28,1.08,679.75,1024.54,680.83
77,2025-03-29,0.00,11.18,256.03,11.18


In [5]:
df_resumo_total02['dia'] = df_resumo_total02['Data'].dt.day
df_resumo_total02['mes'] = df_resumo_total02['Data'].dt.month
df_resumo_total02['ano'] = df_resumo_total02['Data'].dt.year

df_resumo_total02.head()


,Data,Pedidos concluidos,Pedidos_pendentes,Forno,Total Pedidos,dia,mes,ano
0,2025-01-02,118.45,0.0,0.00,118.45,2,1,2025
1,2025-01-03,277.52,0.0,0.00,277.52,3,1,2025
2,2025-01-06,903.08,0.0,653.87,903.08,6,1,2025
3,2025-01-07,929.60,0.0,765.50,929.60,7,1,2025
4,2025-01-08,982.51,0.0,416.88,982.51,8,1,2025


In [6]:
df_concluido['dia_n'] = df_concluido['Conclusão'].dt.day
df_concluido['mes_n'] = df_concluido['Conclusão'].dt.month
df_concluido['ano_n'] = df_concluido['Conclusão'].dt.year



In [7]:
df_concluido.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12293 entries, 0 to 12292
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Cliente            12293 non-null  object        
 1   Pedido             12293 non-null  object        
 2   Serviço            12293 non-null  object        
 3   Status             12293 non-null  object        
 4   Película?          12293 non-null  object        
 5   Vidro              12293 non-null  object        
 6   Pedido Área        12293 non-null  float64       
 7   Pedido Peças       12293 non-null  int64         
 8   Em Produção Área   12293 non-null  int64         
 9   Em Produção Peças  12293 non-null  int64         
 10  Corte Padrão?      5110 non-null   object        
 11  Entrada            12293 non-null  datetime64[ns]
 12  Financ Lib         208 non-null    object        
 13  Técnico Reg        118 non-null    object        
 14  Técnic

In [ ]:
mes = 3
ano = 2025

df_prod_concluido = df_concluido[['dia_n', 'mes_n', 'ano_n', 'tipo_servico', 'Pedido Área']]
prod_dia_filtro = df_prod_concluido[(df_prod_concluido['mes_n']== mes) & (df_prod_concluido['ano_n']== ano)]
df_prod_dia = prod_dia_filtro.groupby('mes_n')['Pedido Área'].sum().reset_index()


In [33]:
total = df_prod_dia['Pedido Área'].iloc[0]
total

np.float64(15840.05)